In [1]:
# https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/

import opendatasets as od

od.download("https://www.kaggle.com/datasets/dawidcegielski/house-prices-in-poland")

# alexandertropin
# ea2bd9c043a1b0dbea99e82a6c45224f

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alexandertropin
Your Kaggle Key: ········


100%|████████████████████████████████████████| 704k/704k [00:00<00:00, 1.73MB/s]

In [66]:
import pandas as pd
import numpy as np

# Preprocessing Data

In [ ]:
'''
Description:

address - Full addres
city - Warszawa (Warsaw), Kraków (Cracow), Poznań (Poznan).
floor - The number of the floor where the apartment is located
id - id
latitude - latitude
longitude - longitude
price - Price of apartment in PLN [TARGET]
rooms - Number of rooms in the apartment
sq - Number of square meters of the apartment
year - Year of the building / apartment
'''

In [6]:
 
file = ('house-prices-in-poland/Houses.csv')
df = pd.read_csv(file, encoding = 'ISO-8859-1')
 
df.head()

,Unnamed: 0,address,city,floor,id,latitude,longitude,price,rooms,sq,year
0,0,Podgórze Zab³ocie Stanis³awa Klimeckiego,Kraków,2.0,23918.0,50.049224,19.970379,749000.0,3.0,74.05,2021.0
1,1,Praga-Po³udnie Grochowska,Warszawa,3.0,17828.0,52.249775,21.106886,240548.0,1.0,24.38,2021.0
2,2,Krowodrza Czarnowiejska,Kraków,2.0,22784.0,50.066964,19.920025,427000.0,2.0,37.00,1970.0
3,3,Grunwald,Poznañ,2.0,4315.0,52.404212,16.882542,1290000.0,5.0,166.00,1935.0
4,4,Ochota Gotowy budynek. Stan deweloperski. Osta...,Warszawa,1.0,11770.0,52.212225,20.972630,996000.0,5.0,105.00,2020.0


In [13]:
df.groupby(['city'])['id'].count()

city
Kraków      9764
Poznañ      4149
Warszawa    9851
Name: id, dtype: int64

In [14]:
price_df = df.drop(['Unnamed: 0','address','id'], axis=1)
# [['city', 'floor','id','latitude','longitude','price','rooms','sq','year']]
price_df.head()

,city,floor,latitude,longitude,price,rooms,sq,year
0,Kraków,2.0,50.049224,19.970379,749000.0,3.0,74.05,2021.0
1,Warszawa,3.0,52.249775,21.106886,240548.0,1.0,24.38,2021.0
2,Kraków,2.0,50.066964,19.920025,427000.0,2.0,37.00,1970.0
3,Poznañ,2.0,52.404212,16.882542,1290000.0,5.0,166.00,1935.0
4,Warszawa,1.0,52.212225,20.972630,996000.0,5.0,105.00,2020.0


In [10]:
price_df.shape

(23764, 9)

In [19]:
print(price_df.isna().sum().sort_values())

city         0
floor        0
latitude     0
longitude    0
price        0
rooms        0
sq           0
year         0
dtype: int64


In [54]:
price_df.dtypes


city          object
floor        float64
latitude     float64
longitude    float64
price        float64
rooms        float64
sq           float64
year         float64
dtype: object

In [55]:
price_df['floor'] = price_df['floor'].astype('int')
price_df['price'] = price_df['price'].astype('int')
price_df['price'] = price_df['price'].astype('int')
price_df['rooms'] = price_df['rooms'].astype('int')
price_df['sq'] = price_df['sq'].astype('int')
price_df['year'] = price_df['year'].astype('int')
price_df2 = price_df.drop(['latitude', 'longitude'], axis=1)
price_df2.head()

,city,floor,price,rooms,sq,year
0,Kraków,2,749000,3,74,2021
1,Warszawa,3,240548,1,24,2021
2,Kraków,2,427000,2,37,1970
3,Poznañ,2,1290000,5,166,1935
4,Warszawa,1,996000,5,105,2020


In [56]:
warsaw_df = price_df2[price_df2['city'] == 'Warszawa']
warsaw_df = warsaw_df.drop(['city'], axis=1).reset_index(drop=True)
warsaw_df.head()

,floor,price,rooms,sq,year
0,3,240548,1,24,2021
1,1,996000,5,105,2020
2,10,2890000,6,280,2003
3,3,615000,4,63,1982
4,1,429000,1,40,1999


In [57]:
krakow_df = price_df2[price_df2['city'] == 'Kraków']
krakow_df = krakow_df.drop(['city'], axis=1).reset_index(drop=True)
krakow_df.head()

,floor,price,rooms,sq,year
0,2,749000,3,74,2021
1,2,427000,2,37,1970
2,2,414600,1,34,2022
3,0,750000,4,81,2021
4,2,280000,1,16,1930


In [58]:
poznan_df = price_df2[price_df2['city'] == 'Poznañ']
poznan_df = poznan_df.drop(['city'], axis=1).reset_index(drop=True)
poznan_df.head()

,floor,price,rooms,sq,year
0,2,1290000,5,166,1935
1,0,421427,3,60,2019
2,8,547000,4,77,2020
3,1,618636,3,66,2020
4,3,563905,4,77,2022


# Linear Regression

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict


In [67]:
X = warsaw_df.drop(['price'], axis=1)
# X.head()
y = np.array(warsaw_df[['price']].astype('int'))
y

array([[ 240548],
       [ 996000],
       [2890000],
       ...,
       [ 360000],
       [ 910000],
       [ 604800]])

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred_simple = reg.predict(X_test)


In [77]:
# R-squared
reg.score(X_test, y_test)

0.24254859574822285

In [92]:
# RMSE

mean_squared_error(y_test, y_pred_simple, squared=False)

605219.4191107464

In [84]:
kf = KFold(n_splits=6, shuffle=True, random_state=42)
reg = LinearRegression()

cv_results = cross_val_score(reg, X, y, cv=kf)
print(cv_results)

[  0.25318022   0.27796125   0.22187527 -66.8688448    0.30597862
   0.25456056]


In [82]:
print(np.mean(cv_results), np.std(cv_results))

-10.925881479498246 25.01846685793459


In [83]:
print(np.quantile(cv_results, [0.025, 0.975]))

[-58.48250479   0.30247645]


In [96]:
y_pred_kf = cross_val_predict(reg, X, y, cv=kf)
# y_pred_kf

In [ ]:
from sklearn.linear_model import Ridge
scores = []
for alpha in [0.1, 1.0, 10.0, 100.0, 1000.0]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    scores.append(ridge.score(X_test, y_test))
print(scores)

In [ ]:
from sklearn.linear_model import Lasso
scores = []
for alpha in [0.01, 1.0, 10.0, 20.0, 50.0]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    lasso_pred = lasso.predict(X_test)
    scores.append(lasso.score(X_test, y_test))
print(scores)

In [ ]:
from sklearn.linear_model import Lasso
X = diabetes_df.drop("glucose", axis=1).values
y = diabetes_df["glucose"].values
names = diabetes_df.drop("glucose", axis=1).columns
lasso = Lasso(alpha=0.1)
lasso_coef = lasso.fit(X, y).coef_
plt.bar(names, lasso_coef)
plt.xticks(rotation=45)
plt.show()

# Linear Classifier (?)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier

In [76]:
from sklearn.metrics import classification_report, confusion_matrix
knn = KNeighborsClassifier(n_neighbors=7)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

        5000       0.00      0.00      0.00         1
      150000       0.00      0.00      0.00         1
      161000       0.00      0.00      0.00         1
      170000       0.00      0.00      0.00         1
      177000       0.00      0.00      0.00         0
      199000       0.00      0.00      0.00         1
      209000       0.00      0.00      0.00         0
      210000       0.00      0.00      0.00         1
      220000       0.00      0.00      0.00         1
      223000       0.00      0.00      0.00         1
      223900       0.00      0.00      0.00         1
      224600       0.00      0.00      0.00         1
      224636       0.17      1.00      0.29         1
      224727       0.50      1.00      0.67         3
      225000       0.50      1.00      0.67         1
      225500   

/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Preci

In [ ]:
 from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {"alpha": np.arange(0.0001, 1, 10),
              "solver": ["sag", "lsqr"]}
ridge = Ridge()
ridge_cv = GridSearchCV(ridge, param_grid, cv=kf)
ridge_cv.fit(X_train, y_train)
print(ridge_cv.best_params_, ridge_cv.best_score_)

In [ ]:
 from sklearn.model_selection import RandomizedSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'alpha': np.arange(0.0001, 1, 10),
              "solver": ['sag', 'lsqr']}
ridge = Ridge()
ridge_cv = RandomizedSearchCV(ridge, param_grid, cv=kf, n_iter=2)
ridge_cv.fit(X_train, y_train)
print(ridge_cv.best_params_, ridge_cv.best_score_)

In [ ]:
 test_score = ridge_cv.score(X_test, y_test)
print(test_score)